# Adding smiles to Database 


In [1]:
import sqlite3
import pandas as pd
from ase.db import connect
import os
from rdkit import Chem
from IPython.display import display
import subprocess
import openbabel
import pybel
from openbabel import pybel
from rdkit.Chem import Draw
from tqdm import tqdm
import json 


/root/miniconda3/envs/my-rdkit-env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Conversion of all mol to smiles using open babel not rdkit 


In [ ]:

# Connect to the database
db = connect('/root/MChem_DGMs/analysis/Databases/JODO_drugs_raw.db')  # Replace with your .db file name

# Create a directory for structures if it doesn't exist
output_dir = '/root/MChem_DGMs/analysis/Databases/structures'
os.makedirs(output_dir, exist_ok=True)

dic={}
# Loop through all rows (molecules) in the database
for row in tqdm(db.select(), desc='loading'):  # Select all rows
    # Get the Atoms object from the current row
    molecule = row.toatoms()
  
    # Write the molecule to an .xyz file in the structures directory
    xyz_filename = os.path.join(output_dir, f'molecule_{row.id}.xyz')  # Save in 'structures'
    molecule.write(xyz_filename)

    # let pybel read file 
    mol = next(pybel.readfile("xyz", xyz_filename))
  
    # Convert to SMILES
    smiles = mol.write("smi").strip()
    smiles = smiles.split('\t')[0].strip()

    dic[f'molecule_{row.id}']=smiles
    



c=1

for i,row in tqdm(enumerate(db.select()),desc='updating'):
        db.update(id=row.id, SMILES= dic[f'molecule_{c}'])
        c=c+1



loading: 0it [00:00, ?it/s]

loading: 1it [00:00,  2.05it/s]==============================
*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is Properties=species:S:1:pos:R:3 pbc="F F F")

loading: 51it [00:00, 114.43it/s]==============================
*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is Properties=species:S:1:pos:R:3 pbc="F F F")

loading: 126it [00:00, 229.33it/s]==============================
*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is Properties=species:S:1:pos:R:3 pbc="F F F")

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is Properties=species:S:1:pos:R:3 pbc="F F F")

loading: 259it [00:01, 375.51it/s]==============================
*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bond

In [9]:
db = connect(f"/root/MChem_DGMs/analysis/Databases/JODO_drugs_raw.db")
smiles_list = []

for row in db.select():
    smiles = row.get("orig_smiles")

    
    smiles_list.append(smiles)

f = open(f"/root/MChem_DGMs/analysis/Databases/JODO_drugs_raw_smiles.json", "w", encoding = "cp1252")
json.dump(smiles_list, f)
f.close()